# Title

Subtitle

## Setup

### Imports & Env. Settings

In [1]:
# Imports
import handcalcs # https://github.com/connorferster/handcalcs/wiki
from handcalcs import render # https://github.com/connorferster/handcalcs?tab=readme-ov-file#basic-usage-1-as-a-jupyter-cell-magic-render
import forallpeople as si # https://connorferster.github.io/forallpeople/basic_usage.html
from modules.section import UniversalColumn
from modules.strength import design_strength

In [2]:
# Environment config.
# --> SI units structural env ( https://connorferster.github.io/forallpeople/environments.html#Environments )
si.environment("structural", top_level=True) # load SI units for structural eng. into the environment, top_level=True defines them in the top-level namespace (as vars)
# -> Handcalcs environment settings for TeX rendering
handcalcs.set_option("latex_block_start", "$")
handcalcs.set_option("latex_block_end", "$")
handcalcs.set_option("math_environment_start", "aligned")
handcalcs.set_option("math_environment_end", "aligned")

### Question Data

In [3]:
%%render params 2 # Loads/Moments, lengths, qn. params
# Actions
F_c = 1 * kN
M_x = 1 * kN * m

# Dimensions
L = 1 * m

# Strengths
p_y = 275 * MPa

<IPython.core.display.Latex object>

### Trial Section

In [4]:
trial_section = UniversalColumn.create(designation="356x368x129")
# ... UB not implemented, should be in rework of sections module

In [5]:
%%render params 1 # BS Notation
D = trial_section.D * mm
B = trial_section.B * mm
d = trial_section.d * mm
t = trial_section.t * mm
T = trial_section.T * mm

S_xx = trial_section.S_xx * 1000*mm**3
S_yy = trial_section.S_yy * 1000*mm**3
Z_xx = trial_section.Z_xx * 1000*mm**3
Z_yy = trial_section.Z_yy * 1000*mm**3
r_xx = trial_section.r_xx * 10*mm
r_yy = trial_section.r_yy * 10*mm
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

In [6]:
%%render params 1 # EC3 Notation
h = D
b = B
t_w = t
t_f = T

W_ply = S_xx
W_plz = S_yy
W_ely = Z_xx
W_elz = Z_yy
i_yy = r_xx
i_zz = r_yy
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

<!-- ### Classification -->

### Effective Length $L_{\text{E}}$ / $L_{\text{Cr}}$

Table 22 BS 5950
_ENV 1993-1-1:2004_

In [13]:
%%render short 2 # Uncomment correct value, defaults to 1.0 L
# L_E = 0.7 * L
# L_E = 0.85 * L
L_E = 1.0 * L
# L_E = 1.2 * L
# L_E = 1.5 * L
# L_E = 2.0 * L

L_Cr = L_E

<IPython.core.display.Latex object>

## BS 5950

### Classification

In [14]:
# Strength class (Defaults to S275) Table 9 BS
thickest = max(trial_section.t, trial_section.T)
p_y = design_strength(thickness=thickest, steel_grade=275)
print(f"Thickness: {thickest} mm")
print(f"Design strength: {p_y} MPa")

Thickness: 17.5 mm
Design strength: 265 MPa


In [15]:
# Section Class Table 11 BS

## Eurocode 3